In [1]:
from NaLLM.data_disambiguation import DataDisambiguation
from NaLLM.unstructured_data_extractor import (
    DataExtractor,
)
from NaLLM.openai import OpenAIChat


In [2]:
from docx import Document as Docx

def read_docx(file_path):
    doc = Docx(file_path)
    full_text = []
    for paragraph in doc.paragraphs:
        full_text.append(paragraph.text)
    return '\n'.join(full_text)

file_path = 'Resources\SongHongDoc.docx'
text = read_docx(file_path)

In [3]:
result = ""

llm = OpenAIChat(
    openai_api_key="sk-", model_name="gpt-3.5-turbo-16k", max_tokens=4000
)

extractor = DataExtractor(llm=llm)
result = extractor.run(data=text)

disambiguation = DataDisambiguation(llm=llm)
disambiguation_result = disambiguation.run(result)

print("Disambiguation result " + str(disambiguation_result))


Starting chunked processing
[{'role': 'system', 'content': '\nYou are a data scientist working for a company that is building a graph database. Your task is to extract information from data and convert it into a graph database.\nProvide a set of Nodes in the form [ENTITY, TYPE, PROPERTIES] and a set of relationships in the form [ENTITY1, RELATIONSHIP, ENTITY2, PROPERTIES]. \nPay attention to the type of the properties, if you can\'t find data for a property set it to null. Don\'t make anything up and don\'t add any extra data. If you can\'t find any data for a node or relationship don\'t add it.\nOnly add nodes and relationships that are part of the schema. If you don\'t get any relationships in the schema only add nodes.\n\nExample:\nSchema: Nodes: [Person {age: integer, name: string}] Relationships: [Person, roommate, Person]\nAlice is 25 years old and Bob is her roommate.\nNodes: [["Alice", "Person", {"age": 25, "name": "Alice}], ["Bob", "Person", {"name": "Bob"}]]\nRelationships: [

In [13]:
print("Disambiguation result " + str(disambiguation_result))

Disambiguation result {'nodes': [{'name': 'Lô 117-121', 'label': 'Bể Sông Hồng', 'properties': {}}, {'name': '[Bể trầm tích Sông Hồng', 'label': 'Bể trầm tích', 'properties': {'diện tích': 'lớn nhất trong số các bể trầm tích Kainozoi chứa dầu khí trên thảm lục địa Việt Nam'}}, {'name': 'Bể Sông Hồng', 'label': 'Bể trầm tích', 'properties': {}}, {'name': 'WA74-pkb', 'label': 'Bể trầm tích', 'properties': {}}, {'name': 'WA-74-shv', 'label': 'Bể trầm tích', 'properties': {}}, {'name': 'Lô 102', 'label': 'Bể trầm tích', 'properties': {}}, {'name': 'Lô 103', 'label': 'Bể trầm tích', 'properties': {}}, {'name': 'Lô 106', 'label': 'Bể trầm tích', 'properties': {}}, {'name': 'Lô 107', 'label': 'Bể trầm tích', 'properties': {}}, {'name': 'BP89', 'label': 'Bể trầm tích', 'properties': {}}, {'name': 'BP91', 'label': 'Bể trầm tích', 'properties': {}}, {'name': 'Lô 102-106', 'label': 'Bể trầm tích', 'properties': {}}, {'name': 'Sông Hồng', 'label': 'Bể trầm tích', 'properties': {}}, {'name': 'Phú K

In [12]:
from langchain.graphs import Neo4jGraph

url = "neo4j+s://d6b3533f.databases.neo4j.io"
username ="neo4j"
password = "1bNOqnTVVqPBzN82Y6DfQoUr0osT-v2b5AUvvfv-Sk8"
graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

# Delete the graph
graph.query("MATCH (n) DETACH DELETE n")

from langchain_community.graphs.graph_document import (
    GraphDocument,
)

graph_document = GraphDocument(
      nodes = [map_to_base_node(node) for node in disambiguation_result["nodes"]],
      relationships = [map_to_base_relationship(rel) for rel in disambiguation_result["relationships"]],
    )
    # Store information into a graph
graph.add_graph_documents([graph_document])

AttributeError: 'str' object has no attribute 'value'